In [1]:
# Importe les librairies
import numpy as np
import pandas as pd
from pathlib import Path
import joblib

<img src='./images/logo-oncfm.png'  width=360px/>

# Détectez des faux billets avec Python<br>Algorithme

> &#9888; Ce *notebook* contient l'algorithme retenu pour la détection des faux billets. Le travail de préparation et de sélection du modèle est présenté dans le fichier `detection-faux-billets-oncfm-preparation-donnees.ipynb`.

## Sommaire
* [Importation du modèle](#1)
* [Choix du fichier à analyser](#2)
* [Vérification des données et détection](#3)
* [Affichage des résultats](#4)

## Importation du modèle<a id='1'></a>

In [7]:
# Définit le modèle à importer et son chemin
model_folder = Path.cwd() / 'modele'
model_file = 'logistic-regression-gs.joblib'
model_path = Path(model_folder / model_file)

# Charge le modèle si le fichier 'joblib' est présent au bon emplacement
if not model_folder.is_dir():
    raise FileNotFoundError(
        f'Impossible de charger le modèle : le dossier \'{model_folder}\''
        f' n\'existe pas.'
    )
elif not model_path.is_file():
    raise FileNotFoundError(
        f'Impossible de charger le modèle : le fichier \'{model_file}\''
        f' est manquant.'
    )
else:
    clf = joblib.load(model_path)
    print('Chargement du modèle effectué avec succès.')

Chargement du modèle effectué avec succès.


## Choix du fichier à analyser<a id='2'></a>

In [8]:
# Initialise le chemin du dossier contenant les fichiers à tester
data_folder = Path.cwd() / 'datasets'

# Initialise l'indice du fichier sélectionné
j=-1

# Vérifie la présence du dossier 'datasets', liste les fichiers CSV qu'il
# contient et récupère le choix utilisateur
if not data_folder.is_dir():
    raise FileNotFoundError(f'Le dossier {data_folder} n\'existe pas.')
else:
    csv_files = [x for x in list(data_folder.glob('*.csv')) if x.is_file()]
    if len(csv_files) != 0:
        print('Fichiers disponibles pour le test :')
        for i, file in enumerate(csv_files):
            print(f'{i} | {file.parts[-1]}')
    else:
        raise FileNotFoundError(
            'Il n\'y a pas de fichiers CSV à tester dans votre dossier.')
    
    while j not in range(0, len(csv_files)):
        j = int(input('Indiquez le numéro du fichier à tester.'))

print(f'\nVous avez choisi le fichier \'{csv_files[j].parts[-1]}\'.')

Fichiers disponibles pour le test :
0 | billets-production-missing-feature.csv
1 | billets-production-no-fake.csv
2 | billets-production-test-nan.csv
3 | billets-production-v2.csv
4 | billets-production.csv

Vous avez choisi le fichier 'billets-production.csv'.


## Vérification des données et détection<a id='3'></a>

In [9]:
# Initialise le set des colonnes que doit contenir le fichier à tester
features = {'diagonal', 'height_left', 'height_right', 'margin_low',
            'margin_up', 'length', 'id'}

# Importe le fichier CSV dans un DataFrame
df = pd.read_csv(csv_files[j])

# Crée un set avec le nom des colonnes du DataFrame
df_features = set(df.columns)

# Vérifie que les features nécessaires sont présentes
if len(features - df_features) != 0:
    raise Exception(
        f'Impossible de tester le fichier, les variables suivantes sont '
        f'manquantes : {list(features - df_features)}')

# Vérifie qu'il n'y a pas de valeurs manquantes.
elif df.isna().any().any():
    raise Exception(
        f'Impossible de tester le fichier, les variables suivantes ont '
        f'des valeurs manquantes : {df.columns[df.isna().any()].to_list()}')

# Effectue la prédiction
else:
    ids = df['id']
    X = df.drop(columns='id')
    y_pred = clf.predict(X)

## Affichage des résultats<a id='4'></a>

In [10]:
# Crée un DataFrame contenant les résultats de la prédiction
df_results = pd.DataFrame(
    zip(ids, y_pred, clf.predict_proba(X)[:, 0], clf.predict_proba(X)[:, 1]),
    columns=['id', 'is_fake', 'prob_0', 'prob_1']
)
df_results['is_fake'] = df_results['is_fake'].map({0: False, 1: True})

df_results['probability_estimates'] = np.where(
    df_results['is_fake'], df_results['prob_1'], df_results['prob_0'])

df_results = df_results.drop(columns=['prob_0', 'prob_1'])

# Affiche les résultats de la détection
fake_banknote_list = list(df_results.loc[df_results['is_fake'] == True, 'id'])

if len(fake_banknote_list) == 0:
    print('Aucun faux billet n\'a été détecté.')
else:
    print(f'Id des faux billets détectés : {fake_banknote_list}\n')
    print('Détails de la détection :\n')
    print(df_results)

Id des faux billets détectés : ['A_1', 'A_2', 'A_3']

Détails de la détection :

    id  is_fake  probability_estimates
0  A_1     True               0.999787
1  A_2     True               0.999977
2  A_3     True               0.999969
3  A_4    False               0.976606
4  A_5    False               0.999984
